# Echo-B 1B4 (Basemodel Training)
This model is a custom 1.4B model containing
- 96 layers
- 1024 embedding size

It was initialized using the original RWKV trainer here : https://github.com/PicoCreator/RWKV-LM-LoRA/blob/picocreator-init-memory-experiment/notebook/echo-B-1B4-training.ipynb

The goal of this experiment, is to measure two seperate objective
- quantify and observe the training of a deep layered model
- test for overfitting conditions across multiple epoch
- test if overfitting is mitigated with the introduction of a noisy state

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

# Basic Setup

In [ ]:
# # First lets get the blank init model, these init model was generated
# # using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
# #
# # As such I have preinitialized these blank models and uploaded them to HF for convinence
# #
# # You can also download the model for each stage respectively
# #
# # Comment back in this cell if you need it
# !mkdir -p ../../../model/
# !mkdir -p ../../../datapath/
# !mkdir -p ../../../checkpoint/
# !rm -rf ../../../model/Echo-B-1B4-Init.pth
# !cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-B-1B4-Init.pth
# !ls -alh ../../../model/Echo-B-1B4-Init.pth

In [ ]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="MultiEpoch-1B4"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

## Stage 1 : Baseline model training

In [ ]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/MultiEpoch-1B4-baseline-enwiki.yaml"

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B4-baseline-enwiki.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Baseline Enwiki Foundation (ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B4-Baseline/last.ckpt" "../model/MultiEpoch-1B4-Baseline.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B4-Baseline.pth"

In [ ]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && python3 dragon_test.py ../model/MultiEpoch-1B4-Baseline.pth "cuda fp32"